### 1. Import Packages

In [1]:
# Librerie di base
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Librerie per la Data Imputation
from sklearn.impute import KNNImputer

# Librerie per la Hyperparameters Optimization
from sklearn.model_selection import GridSearchCV

# Librerie per il Machine Learning
from sklearn import svm
import sklearn.metrics as metrics
from sklearn.model_selection import KFold, train_test_split, cross_val_score

# Librerie per la Features Selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

print('All packages successfully loaded')

All packages successfully loaded


### 2. Load Data & Peak Sheet

In [2]:
df = pd.read_excel('../../data/ST000369.xlsx')

# Rimuoviamo le colonne che non ci servono
df.drop(columns=["Idx", "SampleID", "Class"], inplace=True)

# Visualizziamo le prime 5 righe del dataset
df.head()

,SampleType,HealthState,SmokingStatus,Sex,M1,M2,M3,M4,M5,M6,...,M172,M173,M174,M175,M176,M177,M178,M179,M180,M181
0,Plasma,Adenocarcinoma,Former,F,194,168,77,105985,118,15489,...,195,1606,300,108,7203,43,59,1009,383,68
1,Plasma,Adenocarcinoma,Former,F,215,143,154,100462,133,13534,...,184,364,1364,160,11910,82,99,136,1021,165
2,Plasma,Adenocarcinoma,Current,F,104,67,45,75301,94,7390,...,189,157,884,73,6372,59,35,115,530,72
3,Plasma,Adenocarcinoma,Current,M,360,642,82,42097,84,50943,...,320,1621,461,104,14011,69,86,845,1309,127
4,Plasma,Adenocarcinoma,Current,M,96,137,95,112346,168,41987,...,84,769,266,158,18140,40,87,1213,1037,149


### 2.1 Data Cleaning

Per quanto riguarda la fase di Data Cleaning effettuiamo le stesse operazioni viste per l'algoritmo Random Forest (RF). 

In [3]:
# Effettuiamo la correzione dei valori errati
df["HealthState"] = df["HealthState"].str.replace('Adenocarcnoma', 'Adenocarcinoma')

# Eliminiamo le righe con valori nulli all'interno delle colonne "HealthState" e "Sex"
df = df.dropna(subset=["HealthState", "Sex"])

# Convertiamo i valori di natura categorica in valori numerici
df['Output'] = df['HealthState'].apply(lambda x: 1 if x in ['Adenosquamous', 'Adenocarcinoma'] else 0)
df['SmokingStatus'] = df['SmokingStatus'].apply(lambda x: 1 if x in ['Current'] else 0)
df['Sex'] = df['Sex'].apply(lambda x: 1 if x in ['F'] else 0)

# Visualizziamo le prime 5 righe del dataset
df.head()

,SampleType,HealthState,SmokingStatus,Sex,M1,M2,M3,M4,M5,M6,...,M173,M174,M175,M176,M177,M178,M179,M180,M181,Output
0,Plasma,Adenocarcinoma,0,1,194,168,77,105985,118,15489,...,1606,300,108,7203,43,59,1009,383,68,1
1,Plasma,Adenocarcinoma,0,1,215,143,154,100462,133,13534,...,364,1364,160,11910,82,99,136,1021,165,1
2,Plasma,Adenocarcinoma,1,1,104,67,45,75301,94,7390,...,157,884,73,6372,59,35,115,530,72,1
3,Plasma,Adenocarcinoma,1,0,360,642,82,42097,84,50943,...,1621,461,104,14011,69,86,845,1309,127,1
4,Plasma,Adenocarcinoma,1,0,96,137,95,112346,168,41987,...,769,266,158,18140,40,87,1213,1037,149,1


### 3. Extract X & Y

In [4]:
X = df.drop(columns=['SampleType', 'HealthState', 'Output'])
X_features_names = X.columns
y = df.Output

### 3.1 Data Imputation

In [5]:
# KNNImputer
imputer_knn = KNNImputer(n_neighbors=2)
imputer_knn.fit(X)
X[:] = imputer_knn.fit_transform(X)
X_knn = X.copy()

/var/folders/dv/cqkd8q211lb6w_8jd__s86d40000gn/T/ipykernel_3807/3462717103.py:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X[:] = imputer_knn.fit_transform(X)


### 3.2 Train-test Split

In [6]:
X_train_knn, X_test_knn, y_train_knn, y_test_knn = train_test_split(X_knn, y, test_size=0.2, random_state=42)

### 4. Initial Model Build

In [7]:
# Definiamo il modello SVC con gli iperparametri di default
model = svm.SVC()

# Addestriamo il modello
model.fit(X_train_knn, y_train_knn)

# Eseguiamo le previsioni sui dati di test
y_pred_knn = model.predict(X_test_knn)

### 5. Initial Model Evalutation

In [8]:
# Valutiamo le prestazioni del modello con gli iperparametri di default
accuracy_knn = metrics.accuracy_score(y_test_knn, y_pred_knn)
precision_knn = metrics.precision_score(y_test_knn, y_pred_knn)
recall_knn = metrics.recall_score(y_test_knn, y_pred_knn)
f1_knn = metrics.f1_score(y_test_knn, y_pred_knn)
roc_auc_knn = metrics.roc_auc_score(y_test_knn, y_pred_knn)

print(f'Accuratezza: {accuracy_knn}')
print(f'Precision: {precision_knn}')
print(f'Recall: {recall_knn}')
print(f'F1-score: {f1_knn}')
print(f'ROC AUC: {roc_auc_knn}')

Accuratezza: 0.48484848484848486
Precision: 0.5
Recall: 0.9411764705882353
F1-score: 0.6530612244897959
ROC AUC: 0.47058823529411764


Le prestazioni iniziali sono estremamente deludenti: abbiamo un'accuratezza del 48%, una Precision del 50% e una Recall dell'94%. Proviamo quindi a effettuare l'ottimizzazione dei vari parametri utilizzati per il problema.

### 6. Hyperparameters Optimization

L'**ottimizzazione degli iperparametri** è un passo fondamentale nello sviluppo di modelli predittivi robusti. Infatti, aderire ai parametri predefiniti impedisce ai modelli di raggiungere il massimo delle prestazioni. A tale scopo, utilizziamo la tecnica **Grid Search**.

### 6.1 K-Fold

Prima di effettuare l'ottimizzazione degli iperparametri, cerchiamo il numero di fold ideale per la K-Fold.

In [9]:
# Testiamo diverse configurazioni di K
max = 0
k_best = 0
for k in range(5, 11): 
    kfolds = KFold(n_splits=k, shuffle=True, random_state=42)
    model = svm.SVC()
    scores = cross_val_score(model, X_knn, y, cv=kfolds)
    mean = np.mean(scores)
    if mean > max: 
        max = mean
        k_best = k
    print(f"K={k}, Accuratezza Media: {mean}")

print(f"K ottimale: {k_best}")

K=5, Accuratezza Media: 0.5952651515151515
K=6, Accuratezza Media: 0.601410934744268
K=7, Accuratezza Media: 0.6019668737060042
K=8, Accuratezza Media: 0.5955357142857143
K=9, Accuratezza Media: 0.5955165692007798
K=10, Accuratezza Media: 0.5955882352941176
K ottimale: 7


Il numero ottimale di fold è 7. 
Ora possiamo effettuare la K-fold Cross-Validation con il numero ottimale di fold.


In [10]:
# Creiamo l'oggetto KFold per la Cross-Validation con il numero di fold ottimale
kfolds = KFold(n_splits=k_best, shuffle=True, random_state=42)

# Creiamo il modello SVM con gli iperparametri ottimizzati
model = svm.SVC()

# Applichiamo la K-fold Cross-Validation
scores = cross_val_score(model, X_knn, y, cv=kfolds, scoring='accuracy')

# Visualizziamo i risultati della cross-validation
print(f'Accuratezza media: {scores.mean()}')
print(f'Deviazione standard: {scores.std()}')

Accuratezza media: 0.6019668737060042
Deviazione standard: 0.09357227548423523


### 6.2 KNN Imputator Optimization

Cerchiamo il numero ideale di neighbors per l'imputazione KNN.

In [11]:
imputer_knn = KNNImputer()

# Definiamo il dominio di ricerca
param_space = {'n_neighbors': (1, 20)}

# Creiamo l'oggetto BayesSearchCV
# 'n_iter' determina quanti set distinti di iperparametri verranno esplorati durante la ricerca
# 'cv' indica il numero di fold nell'esecuzione della cross-validation
grid_search = GridSearchCV(imputer_knn, param_space, cv=k_best, scoring='accuracy', n_jobs=-1)

# Addestriamo il modello sui dati
grid_search.fit(X, y)

# Visualizziamo i risultati dell'ottimizzazione bayesiana
print("Migliore configurazione di parametri:", grid_search.best_params_)

KeyboardInterrupt: 

In [ ]:
# KNNImputer
imputer_knn = KNNImputer(n_neighbors=grid_search.best_params_['n_neighbors'])
imputer_knn.fit(X)
X[:] = imputer_knn.fit_transform(X)
X_knn = X.copy()

Abbiamo ottenuto le prestazioni migliori con un numero di fold pari a 19.

### 6.3 Model Hyperparameters

In [ ]:
# Qui di seguito effetuiamo il tuning degli iperparametri del modello

# Creiamo un nuovo modello XGBClassifier
model_2 = svm.SVC()
# GRID SEARCH

# Definiamo la griglia con i parametri da testare

param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': [0.001, 0.01, 0.1, 1, 10],
    'degree': [2, 3, 4],
    'tol': [1e-3, 1e-4, 1e-5],
    'class_weight': [None, 'balanced']
}


# Creiamo l'oggetto BayesSearchCV
bayes_search = GridSearchCV(model_2, param_grid, scoring='accuracy', cv=k_best)

# Eseguiamo la ricerca e cross-validation sui dati di addestramento
scores = cross_val_score(bayes_search, X_knn, y, scoring='accuracy', cv=kfolds)

# Visualizziamo i risultati della cross-validation
for i, score in enumerate(scores):
    print(f'Fold {i+1}: Accuratezza = {score * 100:.2f}%')

# Calcoliamo la media delle accuratezze
mean_accuracy = scores.mean()
print(f'Media delle accuratezze: {mean_accuracy * 100:.2f}%')